# L4: Optimize DSPy Agent with DSPy Optimizer

⏳ **Note (Kernel Starting)**: This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.

**Updated for Groq llama-3.1-8b-instant** - Conservative token usage and latest DSPy documentation

---

💻 **Access files**: Click "File" → "Open" to view `requirements.txt` and `helper.py`

⬇ **Download Notebook**: Click "File" → "Download as" → "Notebook (.ipynb)"

📒 **For more help**, please see the "Appendix – Tips, Help, and Download" Lesson.


In [ ]:
# Cell 2: Setup Groq API Key
from helper import get_groq_api_key
import os

# Load Groq API key from environment
groq_api_key = get_groq_api_key()
os.environ["GROQ_API_KEY"] = groq_api_key

print("✓ Groq API key loaded and set in environment")

In [ ]:
# Cell 3: Configure MLflow Tracking
import mlflow
from helper import get_mlflow_tracking_uri

# Set up MLflow tracking server
mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment("dspy_course_4")

# Enable autologging with full optimization tracking
mlflow.dspy.autolog(
    log_evals=True,           # Log evaluation results
    log_compiles=True,        # Log compilation process
    log_traces_from_compile=True  # Log detailed traces during optimization
)

print("✓ MLflow tracking configured")
print(f"  URI: {mlflow_tracking_uri}")
print(f"  Experiment: dspy_course_4")


In [ ]:
# Cell 4: Configure DSPy with Groq LLM
import dspy

# Configure DSPy to use Groq's llama-3.1-8b-instant
lm = dspy.LM(
    'groq/llama-3.1-8b-instant',
    api_key=groq_api_key,
    max_tokens=512,      # Conservative token usage for RAG answers
    temperature=0.7      # Balance creativity and factual accuracy
)

dspy.configure(lm=lm)

print("✓ DSPy configured with Groq llama-3.1-8b-instant")
print(f"  Model: groq/llama-3.1-8b-instant")
print(f"  Max tokens: 512 (conservative)")
print(f"  Temperature: 0.7")

## Build a RAG Agent

We'll create a Wikipedia-based Retrieval-Augmented Generation (RAG) agent using DSPy's ReAct module.

**ReAct** = Reasoning + Acting - The agent decides when to search for more information before answering.


In [ ]:
# Cell 6: Define Wikipedia Search Tool
def search_wikipedia(query: str) -> list[str]:
    """
    Search Wikipedia abstracts and return top 3 relevant text chunks.
    
    Args:
        query: Search query string
        
    Returns:
        List of text chunks from Wikipedia
    """
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

print("✓ Wikipedia search tool defined")
print("  Data source: Wikipedia abstracts (ColBERTv2)")
print("  Returns: Top 3 relevant chunks per query")


In [ ]:
# Cell 7: Create ReAct Agent
react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

print("✓ ReAct agent created")
print("  Input: question")
print("  Output: answer")
print("  Tools: [search_wikipedia]")
print("  Agent will autonomously decide when to search for information")

## Load Training and Validation Datasets

We'll use a subset of the **HotpotQA** dataset - a question-answering benchmark based on Wikipedia data.

- **Training set**: Used to bootstrap few-shot examples
- **Validation set**: Used to evaluate candidate programs during optimization
- **Dataset size**: Can be as small as 20 records (unlike traditional ML)


In [ ]:
# Cell 9: Load Training Dataset
import json

# Load training set
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        example = dspy.Example(**json.loads(line))
        trainset.append(example.with_inputs("question"))

print(f"✓ Training set loaded: {len(trainset)} examples")

In [ ]:
# Cell 10: Load Validation Dataset

# Load validation set
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        example = dspy.Example(**json.loads(line))
        valset.append(example.with_inputs("question"))

print(f"✓ Validation set loaded: {len(valset)} examples")


In [ ]:
# Cell 11: Inspect Dataset Sample

# Overview of the dataset structure
print("=" * 70)
print("SAMPLE TRAINING EXAMPLE:")
print("=" * 70)
print(trainset[0])
print("=" * 70)
print("\nDataset structure:")
print("  - 'question': Input field")
print("  - 'answer': Expected output (ground truth)")


## Configure MIPROv2 Optimizer

**MIPROv2** (Multi-prompt Instruction Proposal Optimizer v2) automatically optimizes:

1. **Prompt Templates** - Generates comprehensive instructions for each module
2. **Few-Shot Examples** - Bootstraps high-quality demonstrations from training data

### How it works:
1. **Bootstrap** examples by running training data through the program
2. **Generate** instruction candidates using the LLM
3. **Sample** combinations of instructions + few-shot examples
4. **Evaluate** each candidate program on validation set
5. **Select** the best performing configuration

**Auto mode options:**
- `light`: Fast optimization (recommended for getting started)
- `medium`: Balanced optimization
- `heavy`: Thorough optimization (more time/cost)


In [ ]:
# Cell 13: Setup MIPROv2 Optimizer
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,  # How to score program outputs
    auto="light",      # Conservative optimization mode
    num_threads=16     # Parallel evaluation threads
)

print("✓ MIPROv2 Optimizer configured")
print("  Metric: answer_exact_match")
print("  Mode: light (fast, conservative)")
print("  Threads: 16 (parallel evaluation)")
print("\nThe optimizer will:")
print("  1. Bootstrap few-shot examples from training data")
print("  2. Generate instruction candidates via LLM")
print("  3. Evaluate combinations on validation set")
print("  4. Select best performing configuration")


## Load Cache and Run Optimization

⏱️ **Note**: We're using a pre-computed cache (`memory_cache.pkl`) to speed up the process for this demo.

In production, you can run without cache - the optimizer will make real LLM calls.

🔄 **Optimization Process** (typically 5-15 minutes without cache):
- Tests multiple candidate programs
- Continuously tracks and improves scores
- Uses statistical sampling to find optimal configuration


In [ ]:
# Cell 15: Load Memory Cache
dspy.cache.load_memory_cache("./memory_cache.pkl")

print("✓ Memory cache loaded from ./memory_cache.pkl")
print("  This speeds up the demo by avoiding redundant LLM calls")
print("  In production: Remove this line to run fresh optimization")


In [ ]:
# Cell 16: Compile Optimized Program
print("=" * 70)
print("STARTING OPTIMIZATION PROCESS")
print("=" * 70)
print("This will:")
print("  - Bootstrap few-shot examples")
print("  - Generate instruction candidates")
print("  - Evaluate candidate programs")
print("  - Track results in MLflow")
print("\n🔄 Compiling...\n")

optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,  # Skip confirmation prompt
)

print("\n" + "=" * 70)
print("✓ OPTIMIZATION COMPLETE!")
print("=" * 70)


## Inspect Optimized Components

Let's examine what the optimizer changed in our ReAct agent:

### Original Program:
- **Signature**: Simple `question -> answer` with no instructions
- **Demos**: No few-shot examples

### Optimized Program:
- **Signature**: Enhanced with comprehensive instructions
- **Demos**: Bootstrapped few-shot examples showing good reasoning traces


In [ ]:
# Cell 18: View Optimized Signature
print("=" * 70)
print("ORIGINAL SIGNATURE:")
print("=" * 70)
print("question -> answer")
print("(No instructions)")

print("\n" + "=" * 70)
print("OPTIMIZED SIGNATURE:")
print("=" * 70)
print(optimized_react.react.signature)
print("\n✓ Notice the comprehensive instructions added by the optimizer!")


In [ ]:
# Cell 19: View Optimized Demos
print("=" * 70)
print("OPTIMIZED FEW-SHOT EXAMPLES (Demos):")
print("=" * 70)
print(f"\nNumber of demos: {len(optimized_react.react.demos)}")

if optimized_react.react.demos:
    print("\n--- First Demo ---")
    print(optimized_react.react.demos[0])
    print("\n✓ These are bootstrapped examples showing successful reasoning traces")
else:
    print("No demos found")


## Evaluate Performance

Now let's compare the **original** vs **optimized** agent performance on the validation set.

**Metric**: Exact Match - The answer must match the ground truth exactly.

We'll see how much improvement the optimizer achieved without any manual prompt engineering!


In [ ]:
# Cell 21: Setup Evaluator
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,  # Exact match scoring
    devset=valset,          # Evaluation dataset
    display_table=True,     # Show results table
    display_progress=True,  # Show progress bar
    num_threads=24,         # Parallel evaluation
)

print("✓ Evaluator configured")
print(f"  Metric: answer_exact_match")
print(f"  Evaluation set: {len(valset)} examples")
print(f"  Threads: 24")


In [ ]:
# Cell 22: Evaluate Original Agent
print("=" * 70)
print("EVALUATING ORIGINAL REACT AGENT")
print("=" * 70)
print("Running evaluation on validation set...")
print("(This shows baseline performance without optimization)\n")

original_score = evaluator(react)

print("\n" + "=" * 70)
print(f"📊 ORIGINAL SCORE: {original_score:.4f} ({original_score*100:.2f}%)")
print("=" * 70)


In [ ]:
# Cell 23: Evaluate Optimized Agent
print("\n" + "=" * 70)
print("EVALUATING OPTIMIZED REACT AGENT")
print("=" * 70)
print("Running evaluation on validation set...")
print("(This shows performance after MIPROv2 optimization)\n")

optimized_score = evaluator(optimized_react)

print("\n" + "=" * 70)
print(f"📊 OPTIMIZED SCORE: {optimized_score:.4f} ({optimized_score*100:.2f}%)")
print("=" * 70)


In [ ]:
# Cell 24: Performance Comparison
print("\n" + "=" * 70)
print("PERFORMANCE COMPARISON SUMMARY")
print("=" * 70)
print(f"\n{'Metric':<20} {'Original':<15} {'Optimized':<15} {'Improvement':<15}")
print("-" * 70)

original_pct = original_score * 100
optimized_pct = optimized_score * 100
improvement = optimized_pct - original_pct
relative_gain = (improvement / original_pct * 100) if original_pct > 0 else 0

print(f"{'Exact Match Score':<20} {original_pct:>6.2f}%{'':<8} {optimized_pct:>6.2f}%{'':<8} +{improvement:>5.2f}%")
print(f"{'Relative Gain':<20} {'':<15} {'':<15} {relative_gain:>6.1f}%")

print("\n" + "=" * 70)
print("✨ ACHIEVEMENT UNLOCKED: Automatic Prompt Engineering!")
print("=" * 70)
print("\nWithout ANY manual prompt engineering, the MIPROv2 optimizer:")
print(f"  ✓ Generated optimized instructions automatically")
print(f"  ✓ Bootstrapped high-quality few-shot examples")
print(f"  ✓ Improved performance by {improvement:.2f} percentage points")
print(f"  ✓ Achieved {relative_gain:.1f}% relative improvement")
print("\nThis is the power of DSPy Optimization!")
